In this exploration we analyze the demographics and movie preferences of our customer base.  Our larger objective is to connect customers to relevant content in our inventory.  

We may be interested in segmenting our population on any number of features, to enable the use of different strategies to promote content specifically tailored to each segment. Although we will not run a segmentation analysis in this demo, we can begin the process by quickly looking if there are any obvious groups of users based on their demographics.

In [2]:
# Import standard packages

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, cosine

!pip install plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import plotly.plotly as py
tools.set_credentials_file(username='DSOnboardingDemos', api_key='2ETR1b8sWOCWoVoQvpSy')
tools.set_config_file(world_readable=True, sharing='public')

clear_output()

In [3]:
#Install a package, %%capture suppresses installation messages
!sudo apt-get update
!sudo apt-get -y install libsqlite3-dev;
!pip install pyzipcode;

import featurize_helpers.functions as fhf
import plotting_helpers.functions as phf

clear_output()

Get:1 https://deb.nodesource.com/node_6.x xenial InRelease [4,634 B]
Get:2 https://deb.nodesource.com/node_6.x xenial/main Sources [766 B]          
Get:3 https://deb.nodesource.com/node_6.x xenial/main amd64 Packages [963 B]   
Get:4 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]               
Get:5 http://ppa.launchpad.net/openjdk-r/ppa/ubuntu xenial InRelease [17.5 kB] 
Get:6 http://ppa.launchpad.net/openjdk-r/ppa/ubuntu xenial/main amd64 Packages [7,096 B]
Get:7 http://cran.fhcrc.org/bin/linux/ubuntu xenial/ InRelease [3,590 B]       
Get:8 http://cran.fhcrc.org/bin/linux/ubuntu xenial/ Packages [75.2 kB] 
Get:9 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial-security InRelease [102 kB]     
Get:11 http://archive.ubuntu.com/ubuntu xenial/main Sources [1,103 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial/restricted Sources [5,179 B]    
Get:13 http://archive.ubuntu.com/ubuntu xenial/universe Sources [

ImportError: No module named featurize_helpers.functions

# About the Data

In [ ]:
# Load the data
data_dir = ''

users = pd.read_csv(data_dir + 'u.user', sep = '|', names = ['user_id', 'age', 'sex', 'occupation', 'zip_code'],
                    encoding = 'latin-1')

ratings = pd.read_csv(data_dir + 'u.data', sep = '\t', names = ['user_id', 'movie_id', 'rating', 'unix_timestamp'],
                    encoding = 'latin-1')

items = pd.read_csv(data_dir + 'u.item', sep = '|', 
                    names = ['movie id', 'movie title' ,'release date','video release date', 
                             'IMDb URL', 'unknown', 'Action', 'Adventure','Animation', 
                             'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                             'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
                             'Thriller', 'War', 'Western'],
                    encoding = 'latin-1')

# Create Features
users['state'] = users.apply(lambda row: fhf.get_zip_data(row.zip_code, 'state'), axis = 1)

user_features, user_demographics, normed_user_movie_ratings = fhf.get_user_features(users, items, ratings)
user_features['zip_code'] = users['zip_code']
item_features, item_names, item_ids = fhf.get_item_features(items)

In [ ]:
pd.set_option('display.max_columns', 5000)
user_features.head()

In [ ]:
item_features.head()

# Exploratory Analysis

## User Demographics

In [ ]:
traces, layout = phf.user_scatter_plot(user_features)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='user_features')

In [ ]:
trace, layout = phf.user_job_stacked_bar(user_features)
fig = go.Figure(data=trace, layout=layout)
py.iplot(fig, filename='user_features2') 

### Results summary

1.  Fill in an insight in this markup cell

2.  Fill in another insight

# Comparison of inventory volume to average rating for each genre 

In [ ]:
trace0, trace1, layout = phf.inventory_hbar(user_features, item_features)

# Creating two subplots, one for each of the two bar plots defined above
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001, print_grid=False)

# Add the bar plots to the figure
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
py.iplot(fig, filename='movie_genres')

### Results

1.  Drama and Comedy make up most of our library.

2.  Severely lacking in fantasy, animation, westerns, and film-noir.

3.  Users who watched Film-Noir, War, and Documentary movies rated them very highly, but Drama and Comedy movies are lower quality.
        a. We should consider getting more Film-Noir, War, and Documentary movies.
        b. We should consider improving our Drama and Comedy selections.

# Overlap in movie interests

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
phf.conditional_interest(user_features, item_features)

### Results

1.  People who like Children's movies also like animation, but not vice versa.

2.  Fantasy is very polarized.  If people aren't in the group who rates it 4+ stars, they rate it very low across the board.

3.  War and Film-noir are highly rated across all groups.

4.  There is significant overlap in interest between Horror and Thriller.